In [27]:
import warnings  
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder, StandardScaler

from sklearn.model_selection import train_test_split as tts

# Datos


In [2]:
train = pd.read_csv('../csv/csv_limpios/train.csv')

In [3]:
train.head()

,Jugador,Pais,Posc,Equipo,Edad,Nacimiento,PJ,Titular,Min,Goles,Asistencias,Goles+Asistencias,Tarjetas Amarillas,Tarjetas Rojas,Goles Esperados,Asistencias Esperadas,Valor de mercado,jugador_id
0,Aaron Connolly,IRL,Delantero,Brighton,21,2000.00,15.0,8.00,735.00,1.666667,0.666667,2.333333,0.0,0.00,2.80,0.266667,"3,50 mill. €",0
1,Aaron Cresswell,ENG,Defensa,West Ham,33,1989.00,25.6,24.60,2188.60,1.000000,2.400000,3.400000,3.2,0.00,0.76,3.340000,"1,20 mill. €",1
2,Aaron Hickey,SCO,Defensa,Brentford,21,2002.00,20.5,19.00,1551.75,1.250000,0.500000,1.750000,5.5,0.25,0.40,0.600000,"30,00 mill. €",2
3,Aaron Ramsdale,ENG,PO,Arsenal,25,1998.00,30.4,30.40,2736.00,0.000000,0.200000,0.200000,0.6,0.00,0.00,0.160000,"32,00 mill. €",3
4,Aaron Ramsey,ENG,Delantero,Burnley,30,1993.25,14.0,6.75,619.00,1.250000,1.250000,2.500000,1.0,0.00,1.70,1.225000,"2,50 mill. €",4


# Limpieza para generar el modelo

In [4]:
# Seguimos limpieza, convertir valor de mercado a float
def valor_mercado(value):

    value = value.replace('€', '').replace(',', '.').strip()
    parts = value.split()
    
    if len(parts) == 2:
        number, unit = parts
        if unit == 'mill.':
            return float(number) * 1e6
        elif unit == 'mil':
            return float(number) * 1e3
    else:
        return float(value)

In [5]:
train['Valor de mercado'] = train['Valor de mercado'].apply(valor_mercado)

In [6]:
train['Valor de mercado'].value_counts()

2000000.0     155
1000000.0     147
1500000.0     141
3000000.0     135
2500000.0     118
             ... 
950000.0        1
325000.0        1
21000000.0      1
1900000.0       1
375000.0        1
Name: Valor de mercado, Length: 108, dtype: int64

In [7]:
# Veo si hay algun valor de mercado menor a un millon
train[train['Valor de mercado'] == 0.0]

,Jugador,Pais,Posc,Equipo,Edad,Nacimiento,PJ,Titular,Min,Goles,Asistencias,Goles+Asistencias,Tarjetas Amarillas,Tarjetas Rojas,Goles Esperados,Asistencias Esperadas,Valor de mercado,jugador_id
40,Adri Bosch,ESP,Defensa,Granada,22,2001.0,1.0,0.000000,26.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,40
41,Adri Embarba,ESP,Centrocampista,Rayo Vallecano,31,1992.0,21.5,16.166667,1339.333333,1.833333,2.500000,4.333333,3.333333,0.333333,2.916667,3.133333,0.0,41
60,Ahmed Elmohamady,EGY,Defensa,Aston Villa,32,1987.0,16.0,9.500000,935.000000,0.500000,0.500000,1.000000,1.500000,0.000000,0.300000,1.900000,0.0,60
260,Angelo da Costa Júnior,BRA,PO,Bologna,36,1983.0,3.5,3.500000,315.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,260
285,Antonio Iervolino,ITA,Centrocampista,Salernitana,19,2003.0,1.0,0.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3064,Vitorino Antunes,POR,Defensa,Getafe,32,1987.0,21.0,18.666667,1662.000000,0.333333,1.666667,2.000000,6.333333,0.000000,0.900000,2.166667,0.0,3064
3102,William Thomas Fish,ENG,Delantero,Manchester Utd,17,2003.0,1.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,3102
3139,Yeremi Pino,ESP,Delantero,Villarreal,21,2002.0,24.5,16.250000,1349.250000,3.250000,2.750000,6.000000,4.500000,0.000000,3.975000,2.650000,0.0,3139
3167,Álex Cantero,ESP,Delantero,Levante,21,2000.0,14.5,3.500000,423.500000,0.000000,0.500000,0.500000,1.000000,0.000000,1.300000,0.650000,0.0,3167


In [8]:
# Elimino los valores de mercado igual a 0
train = train[train['Valor de mercado'] != 0.0]

In [9]:
# Elimino la columna jugadores ya que solo importan sus estadisticas
train.drop(['Jugador'], axis=1, inplace=True)

In [10]:
train.shape

(3166, 17)

In [11]:
train.head()

,Pais,Posc,Equipo,Edad,Nacimiento,PJ,Titular,Min,Goles,Asistencias,Goles+Asistencias,Tarjetas Amarillas,Tarjetas Rojas,Goles Esperados,Asistencias Esperadas,Valor de mercado,jugador_id
0,IRL,Delantero,Brighton,21,2000.00,15.0,8.00,735.00,1.666667,0.666667,2.333333,0.0,0.00,2.80,0.266667,3500000.0,0
1,ENG,Defensa,West Ham,33,1989.00,25.6,24.60,2188.60,1.000000,2.400000,3.400000,3.2,0.00,0.76,3.340000,1200000.0,1
2,SCO,Defensa,Brentford,21,2002.00,20.5,19.00,1551.75,1.250000,0.500000,1.750000,5.5,0.25,0.40,0.600000,30000000.0,2
3,ENG,PO,Arsenal,25,1998.00,30.4,30.40,2736.00,0.000000,0.200000,0.200000,0.6,0.00,0.00,0.160000,32000000.0,3
4,ENG,Delantero,Burnley,30,1993.25,14.0,6.75,619.00,1.250000,1.250000,2.500000,1.0,0.00,1.70,1.225000,2500000.0,4


#### Vectorizo columnas categoricas

In [12]:
categoricas = train.select_dtypes(include=['object']).columns

In [13]:
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

train_cat = encoder.fit_transform(train[categoricas])

In [14]:
train_cat.shape

(3166, 197)

In [15]:
# Normalizo los datos numericos
numericos = train.select_dtypes(exclude=['object']).columns

# Saco valor de mercado de numericos
numericos = numericos.drop('Valor de mercado')

In [16]:
scaler = StandardScaler()

scaled_numerical_data = scaler.fit_transform(train[numericos])

In [17]:
# Llevo los datos numericos a un dataframe
train_num = pd.DataFrame(scaled_numerical_data, columns=numericos)
# Llevo los datos categoricos a un dataframe
train_cat = pd.DataFrame(train_cat, columns=encoder.get_feature_names_out(categoricas))

In [18]:
# Llevo los numericos y categoricos a un solo dataframe
train_final = pd.concat([train_num, train_cat], axis=1)


In [23]:
# Agrego la columna valor de mercado
train_final['Valor de mercado'] = train['Valor de mercado']

In [38]:
#Contar los nan de valor de mercado
train_final['Valor de mercado'].isna().sum()
#  Los elimino
train_final = train_final.dropna()

## Hacemos el modelo



In [39]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

In [50]:
def model_test(X_train, y_train, X_test, y_test):
    models = {
        "linear_regression": LinearRegression(),
        "ridge_regression": Ridge(),
        "lasso_regression": Lasso(),
        "decision_tree_regressor": DecisionTreeRegressor(),
        "random_forest_regressor": RandomForestRegressor(n_estimators=100),
        "gradient_boosting_regressor": GradientBoostingRegressor(n_estimators=100)
    }

    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        error = mean_squared_error(y_test, y_pred)

        print(f"Model: {name} - MSE: {error}")

In [51]:
X = train_final.drop('Valor de mercado', axis=1)
y = train_final['Valor de mercado']

In [52]:
X_train, X_test, y_train, y_test = tts(X, y, test_size=0.2, random_state=42)

In [53]:
model_test(X_train, y_train, X_test, y_test)

Model: linear_regression - MSE: 4.25897586432698e+33
Model: ridge_regression - MSE: 164984255316036.9
Model: lasso_regression - MSE: 167057356295461.22
Model: decision_tree_regressor - MSE: 532010057459677.44
Model: random_forest_regressor - MSE: 176935149450705.66
Model: gradient_boosting_regressor - MSE: 164876158185559.8


In [55]:
modelo = GradientBoostingRegressor(verbose=False)
modelo.fit(X_train, y_train)

GradientBoostingRegressor(verbose=False)

In [56]:
y_pred = modelo.predict(X_test)

In [58]:
valor = pd.DataFrame(y_pred)
valor.head()

,0
0,6.317394e+06
1,7.132765e+06
2,2.230825e+07
3,9.393459e+06
4,7.955395e+06


In [59]:
valor.shape

(620, 1)